In [10]:
from fastapi import FastAPI
import nest_asyncio
import uvicorn
import pandas as pd 


In [9]:
from flask import Flask, jsonify, request


In [15]:
import pandas as pd

# Leer el archivo Parquet

#df_funcion_developer = pd.read_parquet('Para funcion developer of the year.parquet')

df_funcion_user_for_genre = pd.read_parquet('Para funcion user for genre.parquet')
df_funcion_best_developer_of_the_year = pd.read_parquet('Para funcion developer.parquet')
# Mostrar las primeras filas del DataFrame


In [20]:
df_quees = pd.read_parquet('Para funcion best developer of the year.parquet')

In [21]:
df_quees

,item_id,true_sentiment_from_recommend,sentiment_analysis2,developer,release_date
0,1250,2,2,tripwire interactive,2009-05-14
1,22200,2,2,ace team,2009-04-21
2,227300,2,1,scs software,2013-01-16
3,239030,2,1,3909,2013-08-08
4,248820,2,2,"hopoo games, llc",2013-11-08
...,...,...,...,...,...
53981,70,2,2,valve,1998-11-08
53982,362890,2,2,crowbar collective,2015-05-05
53983,273110,2,1,nexon,2014-10-07
53984,730,2,1,valve,2012-08-21


In [ ]:
df_funcion_best_developer_of_the_year

In [12]:
df_funcion_developer_review = pd.read_parquet('Para funcion developer reviews analisis.parquet')


In [8]:
app = FastAPI()

nest_asyncio.apply()
@app.get("/items-per-year/")
async def items_per_year():
    # Agrupar los datos por año y empresa desarrolladora
    grouped = df_funcion_developer(['year', 'developer']).agg(
        total_items=('items_count', 'sum'),
        free_items=('price_normalized', lambda x: (x == 0).sum())
    ).reset_index()

    # Calcular el porcentaje de items Free
    grouped['free_percentage'] = (grouped['free_items'] / grouped['total_items']) * 100

    # Reorganizar el dataframe para presentarlo en un formato más adecuado
    result = grouped[['year', 'developer', 'total_items', 'free_percentage']]

    # Convertir el dataframe a una lista de diccionarios para el retorno
    result_list = result.to_dict(orient='records')

    return result_list

# Ejecutar FastAPI en Jupyter

uvicorn.run(app, host= "127.0.0.1", port=8080)

INFO:     Started server process [9684]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:56950 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56956 - "GET / HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9684]


In [26]:
@app.get("/top-developers-recommendations/")
async def top_developers_recommendations(year: int):
    # Extraer el año de la columna 'release_date'
    df_quees['release_year'] = df_quees['release_date'].dt.year

    # Filtrar los datos por el año de lanzamiento
    filtered_df = df_quees[df_quees['release_year'] == year]

    if filtered_df.empty:
        return {"error": "No se encontraron datos para el año especificado"}

    # Filtrar solo los juegos que han sido recomendados (true_sentiment_from_recommend == 1)
    recommended_df = filtered_df[filtered_df['true_sentiment_from_recommend'] == 1]

    if recommended_df.empty:
        return {"message": "No se encontraron recomendaciones para el año especificado"}

    # Contar las recomendaciones por desarrollador
    developer_recommendations = recommended_df.groupby('developer')['true_sentiment_from_recommend'].count()

    # Obtener el top 3 de desarrolladores con más recomendaciones
    top_3_developers = developer_recommendations.nlargest(3).reset_index()

    # Convertir el resultado a una lista de diccionarios para devolverlo como JSON
    top_3_list = top_3_developers.to_dict(orient='records')

    return {
        "year": year,
        "top_3_developers": top_3_list
    }

# Ejecutar FastAPI en Jupyter Notebook
nest_asyncio.apply()
uvicorn.run(app, host="127.0.0.1", port=8080)

Task exception was never retrieved
future: <Task finished name='Task-59' coro=<Server.serve() done, defined at C:\Users\h98go\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "C:\Users\h98go\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\main.py", line 577, in run
    server.run()
  File "C:\Users\h98go\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\h98go\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\nest_asyncio.py", line 30, in

INFO:     127.0.0.1:51055 - "GET / HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14548]


In [13]:
app = FastAPI()
@app.get("/developer-sentiment/")
async def developer_sentiment(developer: str):
    # Filtrar el DataFrame por el desarrollador dado
    developer_df = df_funcion_developer_review[df_funcion_developer_review['developer'] == developer]

    if developer_df.empty:
        return {"error": f"No se encontraron reseñas para el desarrollador {developer}"}

    # Contar las reseñas positivas (sentiment_analysis2 == 2)
    positive_count = (developer_df['sentiment_analysis2'] == 2).sum()

    # Contar las reseñas negativas (sentiment_analysis2 == 0)
    negative_count = (developer_df['sentiment_analysis2'] == 0).sum()

    # Crear un diccionario con el nombre del desarrollador como llave
    result = {
        developer: {
            "positive_reviews": positive_count,
            "negative_reviews": negative_count
        }
    }

    return result

# Ejecutar FastAPI en Jupyter Notebook
nest_asyncio.apply()
uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [14548]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50763 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:50772 - "GET /developer-sentiment/?developer=%3Ckoei%3E HTTP/1.1" 200 OK
INFO:     127.0.0.1:50777 - "GET /developer-sentiment/?developer=%3Cvalvei%3E HTTP/1.1" 200 OK
INFO:     127.0.0.1:50778 - "GET /developer-sentiment/?developer=%3Cvalve%3E HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14548]
